# Лабораторная работа №3

ФИО: Тенчурин Александр Андреевич
  
Группа: БИВТ-21-4

  
### План работы:
1. Загрузить необходимые данные к себе и считать (read) их в переменную.
2. Понять, у вас задача классификации (бинарной или многоклассовой) или регрессии.
3. Сделать предобработку данных:  
     1. Разделить выборку на тренировочную (train) и тестовую (test). 
     2. Проверить пропуски в данных. Если они есть, заполнить одной из стратегий, предложенных в ноутбуке для семинара №3. 
     3. Отнормировать численные переменные (`StandardScaler`, `MinMaxScaler`).
     4. Закодировать категориальные признаки по одной из стратегий.
4. Обучить на тренировочном множестве:
     1. Линейную модель (`LogisticRegression`, `LinearRegression`)
     2. Деревянную модель (`DecisionTreeClassifier`, `DecisionTreeRegressor`) 
     3. K-ближайших соседей (`KNeighborsClassifier`, `KNeighborsRegressor`)
     4. Случайный лес (`RandomForestClassifier`, `RandomForestRegressor`) 
5. Посчитайте метрики на train и test множествах:
     1. Для задачи классификации -- Accuracy, ROC-AUC (график + значение), PR-кривую (график), F1-score
     2. Для задачи регрессии -- MAE, RMSE, MAPE
6. Сравните метрики относительно train/test, так и относительно разных моделей. Ответьте на следующие вопросы:
     1. Какая модель справилась лучше с поставленной задачей?
     2. Имеет ли место переобучение?
     3. Имеет ли место недообучение?
     4. Как можно улучшить метрики моделей?

## 1. Импорт данных. Грузим данные

In [ ]:
import pandas as pd 
df = pd.read_csv('student-por.csv') 
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


## 2. Анализ задачи

В целом перед нами стоит задача разработать модель, способную предсказывать успеваемость учащихся на основе доступных данных. Целевая переменная - число, мы хотим спрогнозировать финальную оценку студента, поэтому это задача регрессии. 
Какие основные факторы влияют на результаты тестирования? - Это тоже задача регрессии, где мы исследуем влияние различных факторов (пол, раса, уровень образования родителей, вид обеда, подготовка к экзаменам) на численные результаты тестирования.

In [ ]:
# Вывод информации о типе данных в столбце "G3"
print(df['G3'].dtype)

int64


## 3 Предобработка данных

### A. Разделение выборки на тренировочную (train) и тестовую (test)
Разделение выборки на тренировочную и тестовую позволяет оценить производительность модели на новых, ранее не виденных данных. Это важно для проверки обобщающей способности модели и предотвращения переобучения. Переобучение возникает, когда модель хорошо работает на тренировочных данных, но плохо обобщается на новые данные. В Python, для разделения выборки на тренировочную и тестовую можно использовать функцию train_test_split из библиотеки scikit-learn. Эта функция случайным образом разбивает данные на две части с заданным соотношением. Обычно принято использовать пропорцию 70-80% для тренировочной выборки и 20-30% для тестовой выборки.

In [ ]:
from sklearn.model_selection import train_test_split

# Разделение на матрицу признаков X и целевую переменную y
X = df.drop(columns=['G1', 'G2', 'G3'])  # Удаляем столбцы, которые не являются признаками
y = df[['G1', 'G2', 'G3']]  # Выбираем целевые переменные

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверка размерности выборок (первое число - количество записей, второе - количество используемых столбцов)
print("Размерность тренировочной выборки X_train:", X_train.shape)
print("Размерность тестовой выборки X_test:", X_test.shape)
print("Размерность тренировочной выборки y_train:", y_train.shape)
print("Размерность тестовой выборки y_test:", y_test.shape)
print("")
print("y_train: ")
print(y_train)
print("")
print("y_test: ")
print(y_test)

Размерность тренировочной выборки X_train: (519, 30)
Размерность тестовой выборки X_test: (130, 30)
Размерность тренировочной выборки y_train: (519, 3)
Размерность тестовой выборки y_test: (130, 3)

y_train: 
     G1  G2  G3
332  18  18  18
29   12  11  12
302  10  11  12
286  12  12  13
554  10  11  10
..   ..  ..  ..
71   11   9  10
106  10  10  10
270  14  15  15
435  10  10  10
102  12  13  12

[519 rows x 3 columns]

y_test: 
     G1  G2  G3
636  17  18  19
220  11  11  12
594  18  18  18
429  10  11  11
72   13  11  11
..   ..  ..  ..
514   7   6   7
374  17  18  17
444   9  10  11
244  14  12  12
601   8   8  10

[130 rows x 3 columns]


B. Проверка на пропуски
Следующим шагом идет проверка на пропуски в данных. Исходя из предыдущей лабораторной работы, пропусков в данном датасете нет.

In [ ]:
# Проверка пропусков в данных
print(df.isnull().sum())

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64


### C. Делаем предобработку данных. Отнормирование численных переменных
Следующим шагом необходимо отнормировать численные переменные. Отнормирование численных переменных - это важный шаг в предобработке данных перед построением моделей машинного обучения. Этот процесс заключается в приведении значений численных признаков к одному масштабу или диапазону значений. 

Два основных метода нормирования численных переменных, которые часто используются:

StandardScaler:
Этот метод центрирует данные путем удаления среднего значения каждого признака, а затем масштабирует данные путем деления на стандартное отклонение. Результатом являются признаки со средним значением 0 и стандартным отклонением 1.

MinMaxScaler:
Этот метод преобразует данные так, чтобы они находились в заданном диапазоне, обычно от 0 до 1. Для этого каждое значение признака вычитается из минимального значения признака и затем делится на разницу между максимальным и минимальным значениями признака.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# Создание копии данных для масштабирования
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

# Применение Label Encoding к категориальным переменным
label_encoder = LabelEncoder()
for column in X_train.select_dtypes(include=['object']).columns:
    X_train_encoded[column] = label_encoder.fit_transform(X_train[column])
    X_test_encoded[column] = label_encoder.transform(X_test[column])

# Инициализация и применение StandardScaler
scaler = StandardScaler()
X_train_standart_scaled = scaler.fit_transform(X_train_encoded)
X_test_standart_scaled = scaler.transform(X_test_encoded)
print("X_train_standart_scaled: ")
print(X_train_standart_scaled)
print("")

# Инициализация и применение MinMaxScaler
scaler = MinMaxScaler()
X_train_minmax_scaled = scaler.fit_transform(X_train_encoded)
X_test_minmax_scaled = scaler.transform(X_test_encoded)
print("X_train_minmax_scaled: ")
print(X_train_minmax_scaled)



X_train_standart_scaled: 
[[-0.73799923 -0.87481777  0.98793212 ... -0.21994878 -1.06834489
  -0.78961576]
 [-0.73799923  1.14309521 -0.62953403 ...  2.07844218  1.01613255
   0.06593869]
 [-0.73799923  1.14309521  0.98793212 ... -0.9860791  -0.37351908
  -0.36183854]
 ...
 [-0.73799923  1.14309521 -0.62953403 ... -0.21994878  1.01613255
   0.06593869]
 [ 1.35501497 -0.87481777 -1.4382671  ... -0.21994878  0.32130673
  -0.78961576]
 [-0.73799923  1.14309521 -1.4382671  ... -0.9860791   1.01613255
  -0.36183854]]

X_train_minmax_scaled: 
[[0.         0.         0.42857143 ... 0.25       0.25       0.        ]
 [0.         1.         0.14285714 ... 1.         1.         0.125     ]
 [0.         1.         0.42857143 ... 0.         0.5        0.0625    ]
 ...
 [0.         1.         0.14285714 ... 0.25       1.         0.125     ]
 [1.         0.         0.         ... 0.25       0.75       0.        ]
 [0.         1.         0.         ... 0.         1.         0.0625    ]]


### D Кодировка категориальных признаков
При работе с категориальными признаками в машинном обучении, часто нужно преобразовать их в числовой формат, потому что многие алгоритмы машинного обучения требуют числовых данных для работы.

One-Hot Encoding:
Этот метод создает бинарные (дамми) переменные для каждой категории в категориальном признаке. То есть каждая уникальная категория превращается в новый бинарный признак, который принимает значение 1, если наблюдение относится к этой категории, и 0 в противном случае. One-Hot Encoding предпочтителен, когда нет внутреннего порядка в категориях.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# One-Hot Encoding
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
X_train_one_hot_encoded = one_hot_encoder.fit_transform(X_train)
X_test_one_hot_encoded = one_hot_encoder.transform(X_test)
print("X_train_one_hot_encoded: ")
print(X_train_one_hot_encoded.toarray())
print("")
print("X_test_one_hot_encoded: ")
print(X_test_one_hot_encoded.toarray())
print("")



X_train_one_hot_encoded: 
[[1. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]

X_test_one_hot_encoded: 
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]]



## 4. Обучение на тренировочном множестве

### А. Линейная модель
Для обучения линейной модели на тренировочном множестве используется LogisticRegression для задач классификации или LinearRegression для задач регрессии. В нашем случае необходимо использовать LinearRegression.

In [ ]:
from sklearn.linear_model import LinearRegression

# Инициализация модели
linear_regression_model = LinearRegression()

# Обучение модели на тренировочных данных
linear_regression_model.fit(X_train_one_hot_encoded, y_train)

# Получение предсказаний для тестовых данных
y_pred_linear_regression = linear_regression_model.predict(X_test_one_hot_encoded)
print("y_pred_linear_regression: ")
print(y_pred_linear_regression)

y_pred_linear_regression: 
[[11.15156226 12.29114692 12.55226346]
 [12.78832992 11.59576337 13.39532534]
 [12.46495975 13.51105046 13.20210424]
 [12.31869254 14.20484908 14.02614649]
 [ 8.99078804  8.72389878  9.07401064]
 [13.06108292 13.6575137  13.99724308]
 [13.86988282 13.58818517 14.3802569 ]
 [ 8.07021382  6.42381664  6.4414195 ]
 [10.91903806 10.54534925 11.63902585]
 [11.27824007 12.02418186 12.76136157]
 [13.55432838 13.69392608 14.52134478]
 [11.94700589 11.97678408 11.45789399]
 [13.43644242 13.3744257  13.5609115 ]
 [11.02029234 11.36399841 11.35909386]
 [ 9.75316355  9.31064387  9.03724743]
 [13.68036864 14.30845748 14.85696261]
 [13.06861346 13.87126823 14.39643772]
 [10.38496099 12.95221598 13.85143962]
 [12.79804953 13.40892218 13.61330864]
 [12.79843909 12.55177502 13.49072307]
 [12.6972408  13.2343164  13.5765973 ]
 [11.25798235 10.76682417 11.30668772]
 [13.39298998 13.81015311 15.49526821]
 [11.23051612 11.01300109  9.88233959]
 [11.05849417 10.75406324 10.50519846

### В. Деревянная модель
Для обучения деревянной модели на тренировочном множестве используется DecisionTreeClassifier для задач классификации или DecisionTreeRegressor для задач регрессии. В нашем случае необходимо использовать DecisionTreeRegressor.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Список глубин деревьев
depths = [1, 3, 5, 7, 10]

# Перебор по разным глубинам деревьев
for depth in depths:
    # Инициализация модели с текущей глубиной
    decision_tree_model = DecisionTreeRegressor(max_depth=depth)
    
    # Обучение модели на тренировочных данных
    decision_tree_model.fit(X_train_one_hot_encoded, y_train)
    
    # Оценка производительности на тестовых данных
    score = decision_tree_model.score(X_test_one_hot_encoded, y_test)
    
    # Вывод результатов
    print(f"Глубина дерева: {depth}, Оценка: {score}")


Глубина дерева: 1, Оценка: 0.07989318481378511
Глубина дерева: 3, Оценка: 0.1399610843843722
Глубина дерева: 5, Оценка: 0.030115377321462817
Глубина дерева: 7, Оценка: -0.04579360585223813
Глубина дерева: 10, Оценка: -0.17909159654612647


### С. K-ближайших соседей
Для обучения K-ближайших соседей на тренировочном множестве используется KNeighborsClassifier для задач классификации или KNeighborsRegressor для задач регрессии. В нашем случае необходимо использовать KNeighborsRegressor. Модель k-ближайших соседей имеет гиперпараметр k, который определяет количество ближайших соседей, используемых для прогнозирования.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Список значений k
k_values = [1, 3, 5, 7, 10, 15, 20, 25, 30]

# Перебор по разным значениям k
for k in k_values:
    # Инициализация модели с текущим значением k
    knn_model = KNeighborsRegressor(n_neighbors=k)
    
    # Обучение модели на тренировочных данных
    knn_model.fit(X_train_one_hot_encoded, y_train)
    
    # Оценка производительности на тестовых данных
    score = knn_model.score(X_test_one_hot_encoded, y_test)
    
    # Вывод результатов
    print(f"Значение k: {k}, Оценка: {score}")


Значение k: 1, Оценка: -0.5034475835629469
Значение k: 3, Оценка: -0.10666605529199409
Значение k: 5, Оценка: -0.0009813540578370212
Значение k: 7, Оценка: 0.05651085544423595
Значение k: 10, Оценка: 0.10990983830130345
Значение k: 15, Оценка: 0.1483648356014313
Значение k: 20, Оценка: 0.13547694914344535
Значение k: 25, Оценка: 0.143360517596126
Значение k: 30, Оценка: 0.15353243105627593


### D. Модель случайного леса
Для обучения модели случайного леса на тренировочном множестве используется RandomForestClassifier для задач классификации или RandomForestRegressor для задач регрессии. В нашем случае необходимо использовать RandomForestRegressor.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Инициализация модели
random_forest_model = RandomForestRegressor()

# Обучение модели на тренировочных данных
random_forest_model.fit(X_train_one_hot_encoded, y_train)

# Получение предсказаний для тестовых данных
y_pred_random_forest = random_forest_model.predict(X_test_one_hot_encoded)
print("y_pred_random_forest: ")
print(y_pred_random_forest)

y_pred_random_forest: 
[[11.91 12.34 12.42]
 [11.49 11.75 12.35]
 [12.88 13.38 13.71]
 [11.36 12.21 12.11]
 [11.16 11.45 11.78]
 [12.54 13.02 13.37]
 [14.09 14.24 14.86]
 [ 9.05  9.23  9.41]
 [11.48 11.8  12.22]
 [12.01 12.2  12.6 ]
 [12.27 12.9  12.99]
 [11.65 11.55 12.08]
 [13.1  13.56 14.03]
 [11.35 11.46 11.95]
 [11.13 11.03 11.24]
 [12.04 12.03 12.42]
 [11.96 12.01 12.57]
 [ 9.82 10.46 10.6 ]
 [12.41 12.82 13.37]
 [13.57 13.69 14.25]
 [11.97 12.33 12.59]
 [11.89 12.02 12.23]
 [11.57 11.89 12.41]
 [11.98 12.05 12.26]
 [10.4   9.95  9.71]
 [12.36 12.66 13.02]
 [ 8.86  9.27  8.75]
 [12.45 12.69 13.27]
 [11.9  12.2  12.87]
 [12.61 12.98 13.46]
 [11.6  12.06 12.37]
 [12.59 12.54 13.  ]
 [ 8.91  9.16  9.31]
 [ 8.63  7.36  7.5 ]
 [11.28 11.64 11.04]
 [12.56 12.85 13.56]
 [13.76 13.96 14.43]
 [12.3  12.58 13.11]
 [13.13 13.27 13.67]
 [ 9.24  9.14  9.69]
 [11.24 11.54 11.97]
 [13.21 13.59 14.32]
 [12.91 13.24 13.89]
 [12.76 12.89 13.36]
 [12.63 12.93 13.31]
 [11.15 11.37 11.71]
 [11.94 12.

## 5. Посчитайте метрики на train и test множествах

### А. Подсчет метрик MAE, RMSE, MAPE для задачи регрессии на train и test множествах для модели случайного леса
Для расчета метрик MAE (Mean Absolute Error), RMSE (Root Mean Squared Error) и MAPE (Mean Absolute Percentage Error) на тренировочном и тестовом множествах будут использоваться предсказания модели случайного леса, а затем будет вычислена метрика для данной модели.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Получение предсказаний на тренировочных данных для модели случайного леса
y_pred_random_forest_train = random_forest_model.predict(X_train_one_hot_encoded)

# Получение предсказаний на тестовых данных для модели случайного леса
y_pred_random_forest_test = random_forest_model.predict(X_test_one_hot_encoded)

# Расчет MAE на тренировочном и тестовом множествах для модели случайного леса
train_mae = mean_absolute_error(y_train, y_pred_random_forest_train)
test_mae = mean_absolute_error(y_test, y_pred_random_forest_test)

# Расчет RMSE на тренировочном и тестовом множествах для модели случайного леса
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_random_forest_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_random_forest_test))

# Расчет MAPE на тренировочном и тестовом множествах для модели случайного леса
train_mape = np.mean(np.abs((y_train - y_pred_random_forest_train) / y_train)) * 100
test_mape = np.mean(np.abs((y_test - y_pred_random_forest_test) / y_test)) * 100

# Вывод результатов для модели случайного леса
print("Подсчет метрик для модели случайного леса:")
print("MAE на тренировочном множестве:", train_mae)
print("MAE на тестовом множестве:", test_mae)
print("RMSE на тренировочном множестве:", train_rmse)
print("RMSE на тестовом множестве:", test_rmse)
print("MAPE на тренировочном множестве:", train_mape)
print("MAPE на тестовом множестве:", test_mape)


Подсчет метрик для модели случайного леса:
MAE на тренировочном множестве: 0.689858702633269
MAE на тестовом множестве: 2.030538461538461
RMSE на тренировочном множестве: 0.9294005706264638
RMSE на тестовом множестве: 2.705473511804712
MAPE на тренировочном множестве: inf
MAPE на тестовом множестве: inf


### В. Подсчет метрик MAE, RMSE, MAPE для задачи регрессии на train и test множествах для линейной модели
Для расчета метрик MAE (Mean Absolute Error), RMSE (Root Mean Squared Error) и MAPE (Mean Absolute Percentage Error) на тренировочном и тестовом множествах будут использоваться предсказания линейной модели, а затем будет вычислена метрика для данной модели.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Load the dataset
df = pd.read_csv('student-por.csv')

# Separate features (X) and target variables (y)
X = df.drop(columns=['G1', 'G2', 'G3'])  
y = df[['G1', 'G2', 'G3']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# One-Hot Encoding
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
X_train_one_hot_encoded = one_hot_encoder.fit_transform(X_train)
X_test_one_hot_encoded = one_hot_encoder.transform(X_test)

# Initialize and train the linear regression model
linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train_one_hot_encoded, y_train)

# Make predictions on training and test sets
y_pred_train = linear_regression_model.predict(X_train_one_hot_encoded)
y_pred_test = linear_regression_model.predict(X_test_one_hot_encoded)

# Calculate evaluation metrics
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
train_mape = np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100
test_mape = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

# Print evaluation metrics
print("Подсчет метрик для задачи регрессии:")
print("MAE на тренировочном множестве:", train_mae)
print("MAE на тестовом множестве:", test_mae)
print("RMSE на тренировочном множестве:", train_rmse)
print("RMSE на тестовом множестве:", test_rmse)
print("MAPE на тренировочном множестве:", train_mape)
print("MAPE на тестовом множестве:", test_mape)


Подсчет метрик для задачи регрессии:
MAE на тренировочном множестве: 1.6010082936474959
MAE на тестовом множестве: 2.1957943894956924
RMSE на тренировочном множестве: 2.124420847845465
RMSE на тестовом множестве: 2.8282709599482674
MAPE на тренировочном множестве: inf
MAPE на тестовом множестве: inf


## 6. Сравнение метрик

### А. Сравнение метрики относительно train/test и разных моделей. Модель случайного леса

Результаты метрик предоставляют информацию о том, насколько хорошо модель справляется с прогнозированием целевой переменной на тренировочном и тестовом множествах данных. Интерпретация результатов для модели случайного леса:

MAE (средняя абсолютная ошибка):
- MAE на тренировочном множестве: 0.689858702633269
- MAE на тестовом множестве: 2.030538461538461

MAE измеряет среднее абсолютное отклонение прогнозов модели от фактических значений целевой переменной. Чем ниже значение MAE, тем лучше модель. В данном случае MAE на тренировочном множестве заметно ниже, модель переобучена.

RMSE (корень из среднеквадратичной ошибки):
 - RMSE на тренировочном множестве: 0.9294005706264638
 - RMSE на тестовом множестве: 2.705473511804712

RMSE является средним квадратным корнем из среднеквадратичной ошибки, и он тоже измеряет среднее отклонение прогнозов модели от фактических значений целевой переменной. Чем меньше значение RMSE, тем лучше модель. Как и в случае с MAE, значение RMSE на тренировочном множестве значительно ниже, имеет место переобучение модели.

MAPE (средняя абсолютная процентная ошибка):
 - MAPE на тренировочном множестве: inf
 - MAPE на тестовом множестве: inf

MAPE выражается в процентах и измеряет среднюю абсолютную процентную ошибку прогнозов модели от фактических значений целевой переменной. Значение MAPE равное "inf" указывает на то, что модель абсолютно точно предсказывает некоторые значения на тренировочном множестве. 

Модель случайного леса демонстрирует значительное переобучение, что видно по заметной разнице в значениях MAE и RMSE между тренировочным и тестовым множествами.  Переобучение модели возникает, когда модель слишком хорошо подстроилась под тренировочные данные и потеряла способность обобщать на новые, ранее не встречавшиеся данные. Это является нежелательным явлением, так как модель может давать неправильные прогнозы для новых данных, что снижает ее практическую полезность.

### В. Сравнение метрики относительно train/test и разных моделей. Линейная модель

Результаты метрик предоставляют информацию о том, насколько хорошо модель справляется с прогнозированием целевой переменной на тренировочном и тестовом множествах данных. Интерпретация результатов для линейной модели:

MAE (Mean Absolute Error, средняя абсолютная ошибка):
 - MAE на тренировочном множестве: 1.6010082936474959
 - MAE на тестовом множестве: 2.1957943894956924

MAE измеряет среднее абсолютное отклонение прогнозов модели от фактических значений целевой переменной. Разница в MAE между тренировочным и тестовым множествами не слишком велика, что может быть признаком того, что модель не сильно переобучена, но всё же есть пространство для улучшения.


RMSE (Root Mean Squared Error, корень из среднеквадратичной ошибки):
 - RMSE на тренировочном множестве: 2.124420847845465
 - RMSE на тестовом множестве: 2.8282709599482674

RMSE является средним квадратным корнем из среднеквадратичной ошибки, и он тоже измеряет среднее отклонение прогнозов модели от фактических значений целевой переменной. Более высокое значение RMSE на тестовом множестве указывает на то, что модель имеет трудности с обобщением. Разница в RMSE между тренировочным и тестовым множествах больше, чем разница в MAE, что подтверждает наличие некоторого переобучения.


MAPE (Mean Absolute Percentage Error, средняя абсолютная процентная ошибка):
 - MAPE на тренировочном множестве: inf
 - MAPE на тестовом множестве: inf

MAPE выражается в процентах и измеряет среднюю абсолютную процентную ошибку прогнозов модели от фактических значений целевой переменной. Значение MAPE равное "inf" указывает на то, что модель абсолютно точно предсказывает некоторые значения.

В целом, линейная модель демонстрирует приемлемую точность на тренировочных данных, но показывает признаки переобучения, что подтверждается разницей в значениях MAE и RMSE между тренировочным и тестовым множествах. Проблема с MAPE требует проверки данных и, возможно, использования альтернативных метрик для более точной оценки модели.

### С. Какая модель лучше справилась с поставленной задачей?
У модели случайного леса разброс между тренировочным и тестовым множеством оказался больше, что говорит о большем переобучении модели. Линейная модель луше подходит для данной ситуации.

### D. Имеет ли место переобучение?
Да, переобучение разной степени возникает при выполнении пятого пункта.

### Е. Имеет ли место недообучение?
Нет, признаков недообучения не наблюдалось.

### F. Как можно улучшить метрики моделей?
Улучшение метрик моделей может быть достигнуто путем использования различных стратегий и техник:

- Регуляризация: 
Введение регуляризации (например, L1 или L2) может уменьшить переобучение, добавляя штраф за сложность модели и предотвращая её излишнее подстраивание под тренировочные данные.
- Увеличение объема данных: Дополнительные данные могут помочь модели лучше обобщать закономерности в данных. Нужно увеличить объем тренировочных данных, если это возможно.
 - Использование более сложных моделей: В некоторых случаях простые модели могут быть недостаточно гибкими для описания сложных закономерностей в данны
 